In [1]:
import matplotlib

# Memory leak with Matplotlib when in interactive mode and writing 1000+ plots
matplotlib.use('agg')
matplotlib.interactive(False)

import obspy, os, glob

CSV = 'Benz_catalog.csv' # 'OK_2014-2015-2016.csv' # 
FOLDER_NAME = 'benz_monthly'

# In seconds
DURATION = 20 

# Seconds before event
PRE_PADDING = 10

# Seconds after event
POST_PADDING = 10

DATA_PATH = os.path.join(os.getcwd(), 'data')
SPECTROGRAM_PATH = os.path.join(os.getcwd(), f'spectrograms/{FOLDER_NAME}')

stream_paths = glob.glob(os.path.join(DATA_PATH, 'train_mseed/*.mseed'))

In [2]:
# IMPORT SEISMIC TOOLBOX CODE
import sys
sys.path.insert(0, '/data/seismic_toolbox')

In [3]:
import seismic_code.spectrograms as spectro
import seismic_code.filter as filt
from seismic_code import helpers
from seismic_code.spectrograms import write_spectrogram

In [4]:
from pathlib import Path
from datetime import date
from collections import defaultdict
from obspy import read

class StreamPath:
    def __init__(self, path):
        self.raw_path = path
        self.path = Path(path)
        self.station = self.path.parts[-1].split('_')[0]
        date_str = self.path.parts[-1].split('_')[1].replace('.mseed', "")
        month, year = date_str.split('-')
        self.date_start = date(int(year), int(month), day=1)
        
    def load(self):
        """ Loads a stream from the path """
        return read(self.raw_path)
        
    def __str__(self):
        return self.raw_path
    
paths = list(map(StreamPath, stream_paths))
s1_date_paths = {stream.date_start: stream for stream in paths if stream.station == 'GSOK029'}

In [5]:
from obspy import Stream
from operator import add
from functools import reduce
from obspy import read

# Combine all streams into one
# stream = reduce(add, map(read, stream_paths))

In [6]:
import pandas as pd

quake_csv = pd.read_csv(os.path.join(DATA_PATH, CSV))

In [7]:
def change_column_name(df, column_index, new_name):
    columns = df.columns.values
    columns[column_index] = new_name
    df.columns = columns
    
change_column_name(quake_csv, 0, 'EventNum')

quake_csv.head()

,EventNum,Date,Time,Magnitude,Variance,origintime,utc_timestamp
0,0,02/15/2014,00:02:41,0.37,0.95,2014-02-15T00:02:41.000000Z,1.392423e+09
1,1,02/15/2014,00:03:45,-0.44,0.86,2014-02-15T00:03:45.000000Z,1.392423e+09
2,2,02/15/2014,00:08:07,-0.18,0.93,2014-02-15T00:08:07.000000Z,1.392423e+09
3,3,02/15/2014,00:12:52,0.10,0.93,2014-02-15T00:12:52.000000Z,1.392423e+09
4,4,02/15/2014,00:14:09,-0.47,0.89,2014-02-15T00:14:09.000000Z,1.392423e+09


In [8]:
from itertools import islice
from obspy import UTCDateTime

def gen_row_date(df):
    for index, row in df.iterrows():
        year, month, day = row.origintime[:10].split('-')
        yield date(int(year), int(month), day=1), row
        
        
def gen_filter_waves(df, date_paths, pre_padding=PRE_PADDING, post_padding=POST_PADDING):
    curr_date = None
    stream = None

    for dat, row in gen_row_date(df):
        if dat != curr_date:
            try:
                stream_path = date_paths[dat]
            except KeyError:
                print(f"{dat} not in the stream_path")
                continue
                
            stream = stream_path.load()
            curr_date = dat
            
        # Start PRE_PADDING before event_time, end POST_PADDING after event_time
        yield filt.filter_waveform(stream, UTCDateTime(row.origintime), pre_padding, post_padding)
        
def gen_filter_waves_from_times(times, date_paths, pre_padding=PRE_PADDING, post_padding=POST_PADDING):
    curr_date = None
    stream = None
    i = 0
    for time in times:
        dat = date(int(time.year), int(time.month), day=1)
        if dat != curr_date:
            try:
                stream_path = date_paths[dat]
            except KeyError:
                print(f"{dat} not in the stream_path ({i})", end='\r')
                i += 1
                continue
                
            stream = stream_path.load()
            curr_date = dat
            
        # Start PRE_PADDING before event_time, end POST_PADDING after event_time
        yield filt.filter_waveform(stream, time, pre_padding, post_padding)


In [9]:
date_paths = s1_date_paths
amount_quakes = None   # Write all events if None
amount_noise = 10000   # Per month

In [10]:
import warnings

### Create Paths for Each Month

In [11]:
quake_csv['Date'] = pd.to_datetime(quake_csv['Date'])

In [12]:
unique_months = quake_csv['Date'].dt.strftime("%m/%y").unique().tolist()

In [13]:
date_selector = lambda month_year: quake_csv['Date'].dt.strftime("%m/%y") == month_year

In [14]:
# The first month is not complete, so discard
# Delete months you don't want
del unique_months[0]

In [16]:
del unique_months[:3]

In [17]:
unique_months

['06/14', '07/14', '08/14']

### Write Quakes

In [18]:
with warnings.catch_warnings():   
    warnings.simplefilter("ignore")
    
    for month_year in unique_months:
        path = os.path.join(os.path.join(SPECTROGRAM_PATH, month_year.replace('/', '-')), 'local')
        csv = quake_csv[date_selector(month_year)]
        quake_waves = gen_filter_waves(csv, date_paths)
        spectro.async_write_spectrograms(islice(quake_waves, amount_quakes), path, ignoreexceptions=True, write_streams=True)

Writing Files...


KeyboardInterrupt: 

### Write Noise

In [19]:
def get_csv_times(df):
    times = df[['origintime']]
    for row in times.iterrows():
        time = row[1].origintime
        yield UTCDateTime(time)

In [20]:
# Noise times centers around the given time by default... so pad X seconds in each direction
noise_pad = DURATION / 2

In [21]:
with warnings.catch_warnings():   
    warnings.simplefilter("ignore")
    
    for month_year in unique_months:
        path = os.path.join(os.path.join(SPECTROGRAM_PATH, month_year.replace('/', '-')), 'noise')
        csv = quake_csv[date_selector(month_year)]
        
        # Choose noise times only from the month
        csv_month = quake_csv[date_selector(month_year)]
        times = sorted(get_csv_times(csv_month))
        noise_times = helpers.get_noise_times(times_to_exclude=times, 
                                              startafter=times[0], 
                                              endbefore=times[-1],
                                              amount=amount_noise, 
                                              duration=DURATION)
        noise_times = sorted(noise_times)  # important, to make lazy_loading the streams better for the next function        noise_waves = gen_filter_waves_from_times(noise_times, date_paths, noise_pad, noise_pad)
        noise_waves = gen_filter_waves_from_times(noise_times, date_paths, noise_pad, noise_pad)
        
        # Write
        spectro.async_write_spectrograms(noise_waves, path, ignoreexceptions=False, write_streams=True)

Writing Files...

Wrote Files
Writing Files...
2014-07-01 not in the stream_path (9999)
Wrote Files
Writing Files...

Wrote Files


### Clean Up

In [6]:
def clean_up(path):
    """ Removes empty dirs """
    folders = glob.glob(os.path.join(path, '*/*/*/'))
    i = 0
    for folder in folders:
        if not os.listdir(folder):
            os.rmdir(folder)
            
             

clean_up(SPECTROGRAM_PATH)
